In [2]:
from agents import Agent, Runner, trace, Tool, SQLiteSession
from agents.run_context import RunContextWrapper
from agents.mcp import MCPServerStdio, create_static_tool_filter, MCPServerSse
from IPython.display import Markdown, display
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
### Using Third-Party MCP Server 
# async with MCPServerStdio(
#     params={
#         "command": "npx",
#         "args": ["-y", "@modelcontextprotocol/server-filesystem", "./"],
#     },
#     client_session_timeout_seconds=30
# ) as server:
#     with trace("mcp"):
#         agent = Agent(name="Test", instructions="You are a helpful assistant", model="gpt-4o-mini", mcp_servers=[server])
#         result = await Runner.run(agent, "Create a new file test.txt with content 'Hello world'")
#         print(result.final_output)

In [ ]:
## Both Custom and Third-Party Servers Configuration
params = [
    {
        "name": "Pdf master - who can extract text from pdf",
        "command": "uv",
        "args": ["run", "./mcp_servers/pdf-master.py"],
    },
    {
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "./files"],
    }
]

servers = [MCPServerStdio(params=param, client_session_timeout_seconds=30) for param in params]

In [ ]:
# Connecting servers 
for server in servers:
    await server.connect()

# Using MCP Server with Agent
with trace("non_filter_mcp"):
    agent = Agent(name="mympc", instructions="You are a helpful assistant", model="gpt-4o-mini", mcp_servers=servers)
    result = await Runner.run(agent, "extract this pdf './files/example.pdf' and make summary and important notes from extracted text and save it to data.md")
    display(Markdown(result.final_output))

In [6]:
server1 = MCPServerStdio(
    params={
        "name": "Pdf master - who can extract text from pdf",
        "command": "uv",
        "args": ["run", "./mcp_servers/pdf-master.py"],
    },
    client_session_timeout_seconds=30
)
server2 = MCPServerStdio(
    params={
        "name":"File System Server",
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "./files"],
    },
    # tool_filter=create_static_tool_filter(
    #     allowed_tool_names=["read_file", "write_file", "create_directory"],
    #     blocked_tool_names=["move_file"]
    # ),
    client_session_timeout_seconds=30
)

# SSE Server 
server3 = MCPServerSse(params={"url":"http://127.0.0.1:8000/sse"}, client_session_timeout_seconds=30)

### Check tools of server23
await server3.connect()
await server2.connect()
run_context = RunContextWrapper(context=None)
agent = agent = Agent(name="Test", instructions="You are a helpful assistant", model="gpt-4o-mini")
tools = await server3.list_tools(run_context, agent)
print(len(tools))
for tool in tools:
    print(tool.name)


1
pdf_master_tool


Error in sse_reader
Traceback (most recent call last):
  File "/Users/masudrana/projects/learning/openai_agent_sdk/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/Users/masudrana/projects/learning/openai_agent_sdk/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 271, in __aiter__
    async for part in self._httpcore_stream:
  File "/Users/masudrana/projects/learning/openai_agent_sdk/.venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 407, in __aiter__
    raise exc from None
  File "/Users/masudrana/projects/learning/openai_agent_sdk/.venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 403, in __aiter__
    async for part in self._stream:
  File "/Users/masudrana/projects/learning/openai_agent_sdk/.venv/lib/python3.12/site-packages/httpcore/_async/http11.py", line 342, in __aiter__
    raise exc
  File "/Users/masudrana/projects/learning/opena

In [9]:
# Connecting servers 
# for server in [server1, server2]:
#     await server.connect()

# Using MCP Server with Agent
with trace("filtered_mcp"):
    agent = Agent(name="mympc2", instructions="You are a helpful assistant", model="gpt-4o-mini", mcp_servers=[server3])
    result = await Runner.run(agent, "extract ../files/example.pdf' and make summary")
    display(Markdown(result.final_output))

### Summary of the Study: Mobile Devices in English Language Learning

#### Author
Mariusz Kruk, University of Zielona Gora, Poland

#### Abstract
The study explores how advanced learners of English utilize mobile devices (smartphones and tablets) to enhance their language learning. Through semi-structured interviews with 20 students, the study identifies a dual awareness: while some students actively leverage mobile devices for personalized learning, others exhibit a more spontaneous or unstructured approach.

---

### Key Sections

#### 1. Introduction
Mobile devices offer opportunities for individualized, informal language learning. They are prevalent among learners and serve as tools for learner autonomy and technology-mediated learning experiences.

#### 2. Literature Review
The concept of learner autonomy emphasizes self-directed learning. The integration of mobile technologies has transformed language education, allowing learners to access resources anytime and anywhere, fostering independence and collaborative learning.

#### 3. Methodology
- **Participants**: 20 Polish university students at different levels (BA and MA).
- **Data Collection**: Semi-structured interviews aimed at understanding how and why students use mobile devices for learning English.

#### 4. Findings
1. **Usage of Mobile Devices**: Smartphones were the most frequently used devices. Most students reported increased use over time and felt relatively experienced in using them for learning.
   
2. **Reasons for Use**: Convenience and quick access to information were key motivations. Mobile devices aid in organizing study materials and facilitate quick dictionary access.

3. **Resources and Tools**: Students mainly used dictionaries, apps like Duolingo and Google Translate, and online resources for practicing vocabulary.

4. **Learning Contexts**: A significant number used their devices during leisure time for language practice rather than in formal classroom settings. Teachers seldom encouraged mobile device usage in class.

5. **Language Skills**: Vocabulary was the primary focus, with limited engagement in grammar, reading, listening, or speaking practice through mobile devices.

6. **Study Performance**: 75% of participants felt that mobile device usage enhanced their study efficiency and effectiveness, allowing them to devote more time to English learning.

#### 5. Discussion and Conclusions
The findings suggest that while students generally leverage mobile devices for vocabulary learning, their use remains somewhat unstructured during classroom activities. The study calls for educators to integrate mobile device capabilities into teaching practices to enhance learner autonomy and engagement.

#### Limitations
The study had a small, homogenous sample size and was limited to one-time interviews, which could restrict the findings' generalizability.

---

### Keywords
Mobile devices, learner autonomy, English language learning, technology-mediated learning.